# Revealed comparative advantage

This notebook computes revealed comparative advantage (RCA) indices using `ta-es.parquet` and `ta-os.parquet` from `data/interim/trade-accounting/`. Results are saved as `rca.parquet` in `data/final/`.

For country $s$ and sector $i$, the basic formula of the RCA index is:

$$
{RCA}_{(s,i)} = \frac{{Exports}_{(s,i)}/{Exports}_s}{\sum_r {Exports}_{(r,i)}/{Exports}_r}
$$

Exports may be replaced by value added exports to get a value-added-adjusted RCA.

In [88]:
import numpy as np
import pandas as pd
import duckdb

In [89]:
ta_es = 'ta-es.parquet'
ta_os = 'ta-os.parquet'
outputfile = 'rca'

# ta_es = 'ta62-es.parquet'
# ta_os = 'ta62-os.parquet'
# outputfile = 'rca62'

In [90]:
df_es = duckdb.sql(
    f"""
    (
        SELECT 
            table_si.t, table_si.s, 5 AS agg, table_si.i5 AS i, 
            table_si.exports_si, table_s.exports_s, table_i.exports_i, table_all.exports_all,
            table_si.vaxes_si, table_s.vaxes_s, table_i.vaxes_i, table_all.vaxes_all
            
        FROM (
            SELECT t, s, i5, SUM(Exports) AS exports_si, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_si
            FROM read_parquet('../data/trade-accounting/{ta_es}')
            GROUP BY t, s, i5
        ) AS table_si

        JOIN (
            SELECT t, s, SUM(Exports) AS exports_s, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_s
            FROM read_parquet('../data/trade-accounting/{ta_es}')
            GROUP BY t, s
        ) AS table_s
        ON table_si.t = table_s.t AND table_si.s = table_s.s

        JOIN (
            SELECT t, i5, SUM(Exports) AS exports_i, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_i
            FROM read_parquet('../data/trade-accounting/{ta_es}')
            GROUP BY t, i5
        ) AS table_i
        ON table_si.t = table_i.t AND table_si.i5 = table_i.i5

        JOIN (
            SELECT t, SUM(Exports) AS exports_all, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_all
            FROM read_parquet('../data/trade-accounting/{ta_es}')
            GROUP BY t
        ) AS table_all
        ON table_si.t = table_all.t

        GROUP BY 
            table_si.t, table_si.s, table_si.i5, 
            table_si.exports_si, table_s.exports_s, table_i.exports_i, table_all.exports_all,
            table_si.vaxes_si, table_s.vaxes_s, table_i.vaxes_i, table_all.vaxes_all
        
        ORDER BY table_si.t, table_si.s, table_si.i5
    )

    UNION ALL

    (
        SELECT 
            table_si.t, table_si.s, 15 AS agg, table_si.i15 AS i, 
            table_si.exports_si, table_s.exports_s, table_i.exports_i, table_all.exports_all,
            table_si.vaxes_si, table_s.vaxes_s, table_i.vaxes_i, table_all.vaxes_all
            
        FROM (
            SELECT t, s, i15, SUM(Exports) AS exports_si, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_si
            FROM read_parquet('../data/trade-accounting/{ta_es}')
            GROUP BY t, s, i15
        ) AS table_si

        JOIN (
            SELECT t, s, SUM(Exports) AS exports_s, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_s
            FROM read_parquet('../data/trade-accounting/{ta_es}')
            GROUP BY t, s
        ) AS table_s
        ON table_si.t = table_s.t AND table_si.s = table_s.s

        JOIN (
            SELECT t, i15, SUM(Exports) AS exports_i, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_i
            FROM read_parquet('../data/trade-accounting/{ta_es}')
            GROUP BY t, i15
        ) AS table_i
        ON table_si.t = table_i.t AND table_si.i15 = table_i.i15

        JOIN (
            SELECT t, SUM(Exports) AS exports_all, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_all
            FROM read_parquet('../data/trade-accounting/{ta_es}')
            GROUP BY t
        ) AS table_all
        ON table_si.t = table_all.t

        GROUP BY 
            table_si.t, table_si.s, table_si.i15, 
            table_si.exports_si, table_s.exports_s, table_i.exports_i, table_all.exports_all,
            table_si.vaxes_si, table_s.vaxes_s, table_i.vaxes_i, table_all.vaxes_all
        
        ORDER BY table_si.t, table_si.s, table_si.i15
    )

    UNION ALL

    (
        SELECT 
            table_si.t, table_si.s, 35 AS agg, table_si.i, 
            table_si.exports_si, table_s.exports_s, table_i.exports_i, table_all.exports_all,
            table_si.vaxes_si, table_s.vaxes_s, table_i.vaxes_i, table_all.vaxes_all
            
        FROM (
            SELECT t, s, i, SUM(Exports) AS exports_si, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_si
            FROM read_parquet('../data/trade-accounting/{ta_es}')
            GROUP BY t, s, i
        ) AS table_si

        JOIN (
            SELECT t, s, SUM(Exports) AS exports_s, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_s
            FROM read_parquet('../data/trade-accounting/{ta_es}')
            GROUP BY t, s
        ) AS table_s
        ON table_si.t = table_s.t AND table_si.s = table_s.s

        JOIN (
            SELECT t, i, SUM(Exports) AS exports_i, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_i
            FROM read_parquet('../data/trade-accounting/{ta_es}')
            GROUP BY t, i
        ) AS table_i
        ON table_si.t = table_i.t AND table_si.i = table_i.i

        JOIN (
            SELECT t, SUM(Exports) AS exports_all, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxes_all
            FROM read_parquet('../data/trade-accounting/{ta_es}')
            GROUP BY t
        ) AS table_all
        ON table_si.t = table_all.t

        GROUP BY 
            table_si.t, table_si.s, table_si.i, 
            table_si.exports_si, table_s.exports_s, table_i.exports_i, table_all.exports_all,
            table_si.vaxes_si, table_s.vaxes_s, table_i.vaxes_i, table_all.vaxes_all
        
        ORDER BY table_si.t, table_si.s, table_si.i
    )
    """
).df()

In [91]:
df_os = duckdb.sql(
    f"""
    (
        SELECT 
            table_si.t, table_si.s, 5 AS agg, table_si.i5 AS i, 
            table_si.vaxos_si, table_s.vaxos_s, table_i.vaxos_i, table_all.vaxos_all
            
        FROM (
            SELECT t, s, i5, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_si
            FROM read_parquet('../data/interim/trade-accounting/{ta_os}')
            GROUP BY t, s, i5
        ) AS table_si

        JOIN (
            SELECT t, s, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_s
            FROM read_parquet('../data/interim/trade-accounting/{ta_os}')
            GROUP BY t, s
        ) AS table_s
        ON table_si.t = table_s.t AND table_si.s = table_s.s

        JOIN (
            SELECT t, i5, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_i
            FROM read_parquet('../data/interim/trade-accounting/{ta_os}')
            GROUP BY t, i5
        ) AS table_i
        ON table_si.t = table_i.t AND table_si.i5 = table_i.i5

        JOIN (
            SELECT t, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_all
            FROM read_parquet('../data/interim/trade-accounting/{ta_os}')
            GROUP BY t
        ) AS table_all
        ON table_si.t = table_all.t

        GROUP BY 
            table_si.t, table_si.s, table_si.i5, 
            table_si.vaxos_si, table_s.vaxos_s, table_i.vaxos_i, table_all.vaxos_all
        
        ORDER BY table_si.t, table_si.s, table_si.i5
    )

    UNION ALL

    (
        SELECT 
            table_si.t, table_si.s, 15 AS agg, table_si.i15 AS i, 
            table_si.vaxos_si, table_s.vaxos_s, table_i.vaxos_i, table_all.vaxos_all
            
        FROM (
            SELECT t, s, i15, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_si
            FROM read_parquet('../data/interim/trade-accounting/{ta_os}')
            GROUP BY t, s, i15
        ) AS table_si

        JOIN (
            SELECT t, s, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_s
            FROM read_parquet('../data/interim/trade-accounting/{ta_os}')
            GROUP BY t, s
        ) AS table_s
        ON table_si.t = table_s.t AND table_si.s = table_s.s

        JOIN (
            SELECT t, i15, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_i
            FROM read_parquet('../data/interim/trade-accounting/{ta_os}')
            GROUP BY t, i15
        ) AS table_i
        ON table_si.t = table_i.t AND table_si.i15 = table_i.i15

        JOIN (
            SELECT t, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_all
            FROM read_parquet('../data/interim/trade-accounting/{ta_os}')
            GROUP BY t
        ) AS table_all
        ON table_si.t = table_all.t

        GROUP BY 
            table_si.t, table_si.s, table_si.i15, 
            table_si.vaxos_si, table_s.vaxos_s, table_i.vaxos_i, table_all.vaxos_all
        
        ORDER BY table_si.t, table_si.s, table_si.i15
    )

    UNION ALL

    (
        SELECT 
            table_si.t, table_si.s, 35 AS agg, table_si.i, 
            table_si.vaxos_si, table_s.vaxos_s, table_i.vaxos_i, table_all.vaxos_all
            
        FROM (
            SELECT t, s, i, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_si
            FROM read_parquet('../data/interim/trade-accounting/{ta_os}')
            GROUP BY t, s, i
        ) AS table_si

        JOIN (
            SELECT t, s, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_s
            FROM read_parquet('../data/interim/trade-accounting/{ta_os}')
            GROUP BY t, s
        ) AS table_s
        ON table_si.t = table_s.t AND table_si.s = table_s.s

        JOIN (
            SELECT t, i, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_i
            FROM read_parquet('../data/interim/trade-accounting/{ta_os}')
            GROUP BY t, i
        ) AS table_i
        ON table_si.t = table_i.t AND table_si.i = table_i.i

        JOIN (
            SELECT t, SUM(DAVAX1 + DAVAX2 + REX1 + REX2 + REX3) AS vaxos_all
            FROM read_parquet('../data/interim/trade-accounting/{ta_os}')
            GROUP BY t
        ) AS table_all
        ON table_si.t = table_all.t

        GROUP BY 
            table_si.t, table_si.s, table_si.i, 
            table_si.vaxos_si, table_s.vaxos_s, table_i.vaxos_i, table_all.vaxos_all
        
        ORDER BY table_si.t, table_si.s, table_si.i
    )
    """
).df()

In [92]:
df = pd.merge(df_es, df_os)

df['rca'] = (df['exports_si'] / df['exports_s']) / (df['exports_i'] / df['exports_all'])
df['rca_vaxes'] = (df['vaxes_si'] / df['vaxes_s']) / (df['vaxes_i'] / df['vaxes_all'])
df['rca_vaxos'] = (df['vaxos_si'] / df['vaxos_s']) / (df['vaxos_i'] / df['vaxos_all'])

df.to_parquet(f'../data/{outputfile}.parquet', index=False)

In [93]:
df

,t,s,agg,i,exports_si,exports_s,exports_i,exports_all,vaxes_si,vaxes_s,vaxes_i,vaxes_all,vaxos_si,vaxos_s,vaxos_i,vaxos_all,rca,rca_vaxes,rca_vaxos
0,2000,1,5,1,26725.773546,9.197234e+04,6.147901e+05,7.424751e+06,23772.975611,7.767344e+04,5.274769e+05,5.551478e+06,22279.213598,7.767344e+04,5.999530e+05,5.551478e+06,3.509361,3.221189,2.654109
1,2000,1,5,2,14496.867945,9.197234e+04,1.364481e+06,7.424751e+06,12100.858524,7.767344e+04,1.017911e+06,5.551478e+06,9743.948565,7.767344e+04,8.336905e+05,5.551478e+06,0.857692,0.849654,0.835346
2,2000,1,5,3,25527.668603,9.197234e+04,3.564152e+06,7.424751e+06,19413.009003,7.767344e+04,2.429174e+06,5.551478e+06,12158.388764,7.767344e+04,1.651690e+06,5.551478e+06,0.578202,0.571176,0.526119
3,2000,1,5,4,21285.450599,9.197234e+04,1.718435e+06,7.424751e+06,18802.403808,7.767344e+04,1.437800e+06,5.551478e+06,28978.272721,7.767344e+04,2.253499e+06,5.551478e+06,0.999941,0.934654,0.919075
4,2000,1,5,5,3936.581420,9.197234e+04,1.628929e+05,7.424751e+06,3584.191071,7.767344e+04,1.391147e+05,5.551478e+06,4513.614371,7.767344e+04,2.126453e+05,5.551478e+06,1.950930,1.841426,1.517067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55435,2021,63,35,31,19299.751505,3.018193e+06,1.097099e+05,2.668117e+07,15789.133246,2.278304e+06,9.540787e+04,1.907895e+07,20219.148998,2.278304e+06,1.604994e+05,1.907895e+07,1.555120,1.385852,1.054951
55436,2021,63,35,32,15712.480730,3.018193e+06,7.418520e+04,2.668117e+07,13476.766376,2.278304e+06,6.814470e+04,1.907895e+07,13050.457112,2.278304e+06,1.133300e+05,1.907895e+07,1.872343,1.656138,0.964325
55437,2021,63,35,33,13381.860678,3.018193e+06,5.032741e+04,2.668117e+07,10486.539107,2.278304e+06,4.242797e+04,1.907895e+07,10785.001479,2.278304e+06,5.965028e+04,1.907895e+07,2.350552,2.069773,1.514086
55438,2021,63,35,34,19335.567305,3.018193e+06,2.808179e+05,2.668117e+07,15047.074782,2.278304e+06,2.354089e+05,1.907895e+07,27935.857645,2.278304e+06,3.471102e+05,1.907895e+07,0.608682,0.535269,0.673965
